In [37]:
import math

In [63]:
def substitutionSh(word, k, deSh = False, **kwargs):
    delta = -k if deSh else k #определяем отступ
    alphc = ord("Z") - ord("A") + 1 #размер алфавита
    res = ''
    for i in word:
        if not i.isalpha():
            res += i
            continue
        if i.islower(): #сдвигаем элемент по ascii таблице в зависимости от регистра
            res += chr((ord(i) + delta - ord("a")) % alphc + ord("a")) 
        else:
            res += chr((ord(i) + delta - ord("A")) % alphc + ord("A"))
    return res

In [134]:
#Частота букв
ABC = "zqjxkvbwypgumcfldhsirnoate" #1-граммы
ABC = ['th', 'he', 'in', 'er', 'an', 're', 'on', 'at', 'en', 'nd', 'ti', 'es', 'or', 'te', 'of', 'ed', 'is', 'it', 'al', 'ar'] #2-граммы 
frequency = [0.07, 0.11, 0.13, 0.15, 0.4, 0.9, 1.4, 1.5, 1.9, 1.9, 2, 2.4, 2.5, 2.7, 2.9, 3.4, 3.8, 5.2, 6.1, 6.3, 6.8, 7.1, 7.9, 8.1, 10.5, 13] #1-граммы
frequency = [ #2-граммы
3.56,
3.07,
2.43,
2.05,
1.99,
1.85,
1.76,
1.49,
1.45,
1.35,
1.34,
1.34,
1.28,
1.20,
1.17,
1.17,
1.13,
1.12,
1.09,
1.07
]
freq = dict(zip(ABC, [round(i / 100, 4) for i in frequency])) #словарь соответствий

In [81]:
with open("text.txt") as f: #считываем текст из файла
    mainText = f.read()
#mainText = "By definition, IP is an estimate of the sum of the squares of the probabilities of each letter. With a multi-alphabetic substitution for the English language, depending on the number of alphabets used in encryption (m), the IP is expected"

In [141]:
def recognizeStateMachine(func): #декоратор - конечный автомат распознвания слов длины wordLen, func - функция применяемая к распознанному слову
    """
    funcOnNotAlpha - функция на обработку не буквенных символов
    """
    def f(string, wordLen, funcOnNotAlpha = lambda x: x): 
        word = ''
        for i in string:
            if i.isalpha():
                word += i.lower()
                if len(word) == wordLen:
                    word = func(word)
            else:
                funcOnNotAlpha(word + i)
                word = ''
    return f

In [ ]:
def alphaLen(string, wordLen = 1): #подсчет количества слов длины wordLen
    ln = [0]
    @recognizeStateMachine
    def f(word):
        ln[0] += 1
        return word[1:]
    f(string, wordLen)
    return ln[0]
def alphaCount(string, wordLen = 1): #подсчитываем количественные отношения всех слов длины wordLen к общему кол-ву слов
    res = dict({})
    ln = alphaLen(string, wordLen)
    @recognizeStateMachine
    def f(word):
        if word not in res:
            res.update({word : 1 / ln})
        else:
            res[word] += 1 / ln
        return word[1:]
    f(string, wordLen)
    return res

In [42]:
def dictSortByVal(dictt): #сортировка словаря по values
    return dict(sorted(dictt.items(), key= lambda x: x[1], reverse=True))

In [147]:
def sybmolDesh(string, wordLen = 1, freq = freq, markReplace = ''): #дешифровка частотным методом, markReplace - символ для выделения измененных символов
    freq = dictSortByVal(freq) #словарь введенных частот
    allpc = dictSortByVal(alphaCount(string, wordLen)) #словрь подсчитанных в зашифрованном тексте
    replaceDict = dict(zip(allpc.keys(), freq.keys())) # словарь соответвия
    res = ['']
    @recognizeStateMachine #заменяем все слова соответствующие словарю соответствия
    def f(word):
        if word in replaceDict:
            res[0] += markReplace +  replaceDict[word] + markReplace
            return ''
        else:
            res[0] += word[0]
            return word[1:]
    def isNotAlpha(word):
        res[0] += word
    f(string, wordLen, isNotAlpha)
    return res[0]

In [155]:
print(mainText, "\n")
print(substitutionSh(mainText, 2), "\n")
print(sybmolDesh(substitutionSh(mainText, 2), 2), "\n")

Cfqnh Jkvngt ycu cp Cwuvtkcp-dqtp Igtocp rqnkvkekcp yjq ycu vjg fkevcvqt qh Igtocpa htqo 1933 wpvkn jku fgcvj kp 1945. Jg tqug vq rqygt cu vjg ngcfgt qh vjg Pcbk Rctva,[c] dgeqokpi vjg ejcpegnnqt kp 1933 cpf vjgp cuuwokpi vjg vkvng qh HSdjtgt wpf Tgkejumcpbngt kp 1934.[d] Fwtkpi jku fkevcvqtujkr, jg kpkvkcvgf Yqtnf Yct KK kp Gwtqrg da kpxcfkpi Rqncpf qp 1 Ugrvgodgt 1939. Jg ycu enqugna kpxqnxgf kp oknkvcta qrgtcvkqpu vjtqwijqwv vjg yct cpf ycu egpvtcn vq vjg rgtrgvtcvkqp qh vjg Jqnqecwuv, vjg igpqekfg qh cdqwv ukz oknnkqp Lgyu cpf oknnkqpu qh qvjgt xkevkou.

Jkvngt ycu dqtp kp Cwuvtkc-Jwpicta cpf ycu tckugf pgct Nkpb. Jg nkxgf kp Xkgppc ncvgt kp vjg hktuv fgecfg qh vjg 1900u cpf oqxgf vq Igtocpa kp 1913. Jg ycu fgeqtcvgf fwtkpi jku ugtxkeg kp vjg Igtocp Ctoa kp Yqtnf Yct K. Kp 1919, jg lqkpgf vjg Igtocp Yqtmgtu' Rctva (FCR), vjg rtgewtuqt qh vjg Pcbk Rctva, cpf ycu crrqkpvgf ngcfgt qh vjg Pcbk Rctva kp 1921. Kp 1923, jg cvvgorvgf vq ugkbg iqxgtpogpvcn rqygt kp c hckngf eqwr kp Owpkej c